In [1]:
#Import necessary packages
import numpy as np
import pandas as pd
import cv2
import imutils
import tensorflow
import keras
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
from scipy.spatial import distance as dist
import warnings
warnings.simplefilter('ignore')

In [2]:
#All the paths
LabelPath=r"C:\Users\HP\Desktop\yolo-coco\coco.names.txt"
WeightsPath=r"C:\Users\HP\Desktop\yolo-coco\yolov3.weights"
ConfigPath=r"C:\Users\HP\Desktop\yolo-coco\yolov3.cfg.txt"
MIN_CONF=0.3
NMS_THRESH=0.3
MIN_DISTANCE=50
INPUT_VIDEO=r"F:\OBJECT DETECTION\background video _ people _ walking _.mp4"
OUTPUT_VIDEO=r"F:\OBJECT DETECTION\Social distance folder/Output.avi"

In [7]:
#single function definition for detecting people 
def detect_people(frame,net,ln,personIdx=0):
    (h,w)=frame.shape[:2]
    results=[]
    
    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    
    blob=cv2.dnn.blobFromImage(frame,1/255.0,(416,416),swapRB=True,crop=False)
    net.setInput(blob)
    layerOutputs=net.forward(ln)
    
    
    # initialize our lists of detected bounding boxes, centroids, and
    # confidences, respectively
    boxes=[]
    centroids=[]
    confidences=[]
    
    for output in layerOutputs:
        for detection in output:
            scores=detection[5:]
            classid=np.argmax(scores)
            confidence=scores[classid]
            
            if classid==personIdx and confidence>MIN_CONF:
                box=detection[0:4]*np.array([w,h,w,h])
                (centreX,centreY,width,height)=box.astype('int')
                
                x=int(centreX-(width/2))
                y=int(centreY-(height/2))
                
                boxes.append([x,y,int(width),int(height)])
                centroids.append((centreX,centreY))
                confidences.append(float(confidence))
                
    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idx=cv2.dnn.NMSBoxes(boxes,confidences,MIN_CONF,NMS_THRESH)
    
    if len(idx)>0:
        for i in idx.flatten():
            (x,y)=(boxes[i][0],boxes[i][1])
            (w,h)=(boxes[i][2],boxes[i][3])
            
            r=(confidences[i],(x,y,x+w,y+h),centroids[i])
            results.append(r)
            
    return results
            
    

In [8]:
Labels=open(LabelPath).read().strip().split('\n')
args={'display':1}

# load our YOLO object detector trained on COCO dataset (80 classes)
net=cv2.dnn.readNetFromDarknet(ConfigPath,WeightsPath)

ln=net.getLayerNames()
ln=[ln[int(i)-1] for i in net.getUnconnectedOutLayers()]


# initialize the video stream and pointer to output video file
vs=cv2.VideoCapture(INPUT_VIDEO if INPUT_VIDEO else 0)
writer=None

while True:
    (grabbed,frame)=vs.read()
    if not grabbed:
        break
    frame=imutils.resize(frame,width=700)
    results=detect_people(frame,net,ln,personIdx=Labels.index('person'))
    
    violate=set()
    
    # ensure there are *at least* two people detections (required in
    # order to compute our pairwise distance maps)
    if len(results)>2:
        centroid=np.array([r[2] for r in results])
        D=dist.cdist(centroid,centroid,metric='euclidean')
        
        
        # loop over the upper triangular of the distance matrix
        for i in range(0,D.shape[0]):
            for j in range(i+1,D.shape[1]):
                
                # check to see if the distance between any two
                # centroid pairs is less than the configured number
                # of pixels
                
                if D[i,j]<MIN_DISTANCE:
                    violate.add(i)
                    violate.add(j)
                    
    # loop over the results              
    for(i,(probs,boxes,centroids)) in enumerate(results):
        # extract the bounding box and centroid coordinates, then
        # initialize the color of the annotation
        
        (startX,startY,endX,endY)=boxes
        (cx,cy)=centroids
        colors=(0,255,0)
        
        
        # if the index pair exists within the violation set, then
        # update the color
        
        if i in violate:
            colors=(0,0,255)
            
            cv2.rectangle(frame,(startX,startY),(endX,endY),colors,2)
            cv2.circle(frame,(cx,cy),5,colors,1)
            
        text='Social Distance Violated:{}'.format(len(violate))
        cv2.putText(frame,text,(10,frame.shape[0]-25),cv2.FONT_HERSHEY_SIMPLEX,0.85,(0,0,255),2)
        
        if args['display']>0:
            cv2.imshow('Output',frame)
            key=cv2.waitKey(1) & 0xFF
            if key==ord('q'):
                break
        if OUTPUT_VIDEO!='' and writer is None:
            
            # initialize our video writer
            fourcc=cv2.VideoWriter_fourcc(*'MJPG')
            writer=cv2.VideoWriter(OUTPUT_VIDEO,fourcc,25,(frame.shape[0],frame.shape[1]),True)
            
        
        # if the video writer is not None, write the frame to the output
        # video file
        
        if writer is not None:
            writer.write(frame)
        
                    